In [1]:
from figconvnet.figconvunet import FIGConvUNet
from figconvnet.geometries import GridFeaturesMemoryFormat
import torch
import warp as wp


Warp 1.6.2 initialized:
   CUDA Toolkit 12.8, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A100-SXM4-80GB" (79 GiB, sm_80, mempool enabled)
     "cuda:1"   : "NVIDIA A100-SXM4-80GB" (79 GiB, sm_80, mempool enabled)
     "cuda:2"   : "NVIDIA A100-SXM4-80GB" (79 GiB, sm_80, mempool enabled)
     "cuda:3"   : "NVIDIA A100-SXM4-80GB" (79 GiB, sm_80, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/ubuntu/.cache/warp/1.6.2


In [2]:
device = "cuda:2"
torch_device = torch.device(device)
gpu_index = torch_device.index
torch.cuda.set_device(gpu_index)

In [3]:
print(device)
print(torch_device)
print(gpu_index)

cuda:2
cuda:2
2


In [4]:
model = FIGConvUNet(
        in_channels = 2,
        out_channels = 1,
        kernel_size = 3, 
        hidden_channels = [8, 16, 32],
        mlp_channels = [16, 16],
        num_levels = 2,
        resolution_memory_format_pairs = [
            (GridFeaturesMemoryFormat.b_xc_y_z, (2, 16, 16)),   
            (GridFeaturesMemoryFormat.b_yc_x_z, (16, 2, 16)),
            (GridFeaturesMemoryFormat.b_zc_x_y, (16, 16, 2)),
        ],
        aabb_max = [1.0, 1.0, 1.0],
        aabb_min = [0.0, 0.0, 0.0] 
        ).to(torch_device)

/data/envs/physicsnemo/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
vertices = torch.rand(1, 100, 3, device=torch_device)
features = torch.rand(1, 100, 2, device=torch_device)

In [6]:
warp_dev = wp.get_cuda_device(gpu_index)

with wp.ScopedDevice(warp_dev):
    pred, _ = model(vertices, features)

Module figconvnet.warp_neighbor_search 64995f3 load on device 'cuda:2' took 2.12 ms  (cached)


In [7]:
print(pred.shape)

torch.Size([1, 100, 1])
